In [ ]:
# TODO: take away X,Y,Z from dataset and retrain model
# Work on getting confidence of each pick

In [194]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from PIL import Image
import tensorflow as tf

In [19]:
img = cv2.imread("INPUT_TEST_IMAGE.jpg")

In [35]:
def adjustBrightness(image, brightness_factor):
    # scale pixel values by brightness factor
    return cv2.addWeighted(image, 1 + brightness_factor, image, 0, 0)

In [37]:
def adjustContrast(image, contrast_factor):
    # scale pixel values by contrast factor
    mean = cv2.mean(image)[0]
    return cv2.addWeighted(image, contrast_factor, image, 0, mean * (1 - contrast_factor))

In [142]:
def preprocessImage(img):
  grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  grayImg = cv2.GaussianBlur(grayImg, (5, 5), 0)
  brightnedImg = adjustBrightness(grayImg, 2.5)
  contrasted_gray = adjustContrast(brightnedImg, 2.0)

  # Method 1
  # filtered = cv2.bilateralFilter(contrasted_gray, 11, 17, 17)
  # thresh = cv2.adaptiveThreshold(contrasted_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
  # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  # cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

  # Method 2
  ret, thresh = cv2.threshold(contrasted_gray, 200, 255, cv2.THRESH_BINARY)
  invertedThresh = cv2.bitwise_not(thresh)
  cv2_imshow(invertedThresh)

  return invertedThresh

In [ ]:
processedImg = preprocessImage(img)

In [166]:
def segmentImage(img):
  digits = []
  contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # for cnt in contours:
  #   x, y, w, h = cv2.boundingRect(cnt)
  #   if w > 10 and h > 10: # Filter out small noise contours
  #       # Extract the digit and save as a separate image

  #       digit = img[y:y+h, x:x+w]
  #       digits.append(digit)
  #       cv2_imshow(digit)
  #       # cv2.imwrite('digit_{}.jpg'.format(x), digit)

  for contour in contours:
    # Get the bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    
    # Crop the bounding rectangle from the original image
    digit = img[y:y+h, x:x+w]
    digits.append(digit)

    # removing images that are too small
    finalDigits = []
    for i in range(len(digits)):
        if np.shape(digits[i])[0] > 16:
          finalDigits.append(digits[i])

  return finalDigits

In [167]:
digits = segmentImage(processedImg)

In [172]:
model = tf.keras.models.load_model('DigitClassifier.h5')

In [207]:
def predictValues(digits):
  predictions = []
  accuracies = []
  for digit in digits:
    image = Image.fromarray(digit)
    resized_image = image.resize((64, 64))
    numpy_image = asarray(resized_image)
    finalArray = np.reshape(numpy_image, (1, 64, 64, 1))
    # final = np.expand_dims(resized_image, axis=-1) # Add a channel dimension
    # print(final.shape)

    prediction = model.predict(finalArray)
    predictions.append(prediction)
  return predictions

In [208]:
predictions = predictValues(digits)

1/1 [==============================] - 0s 26ms/step


In [213]:
def getIndexMappingPrediction(predicted_class_index):
  string_to_index = {
        "0": 0,
        "1": 1,
        "2": 2,
        "3": 3,
        "4": 4,
        "5": 5,
        "6": 6,
        "7": 7,
        "8": 8,
        "9": 9,
        "add": 10,
        "dec": 11,
        "eq": 12,
        "div": 13,
        "mul": 14,
        "sub": 15,
        "x": 16,
        "y": 17,
        "z": 18,
    }
  index_to_string = {v: k for k, v in string_to_index.items()}
  predicted_class_label = index_to_string[predicted_class_index]
  return predicted_class_label

In [214]:
for prediction in predictions:
  predicted_class_index = np.argmax(prediction)
  label = getIndexMappingPrediction(predicted_class_index)
  print(label)

8
8
z
x
